In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os        
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
 #Preparing the training data using preprocessing of the images


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen =ImageDataGenerator(rescale=1./255, validation_split=0.26)
    
train_seedlings = train_datagen.flow_from_directory(
        '../input/plant-seedlings-classification/train',  
            target_size=(224, 224),  # Resizes images
            batch_size=80,
            class_mode='categorical',subset = 'training', seed=50)

x_train, y_train = next(train_seedlings)

In [ ]:
#Setting up validation Data (train/test split)

validation_seedlings = train_datagen.flow_from_directory(
    '../input/plant-seedlings-classification/train',
    target_size=(224, 224),
    batch_size=82,
    class_mode='categorical',
    subset='validation')

In [ ]:
#Let's see what some examples of seedlings in our training set look like

import matplotlib.pyplot as plt
images = x_train[:9]
labels = y_train[:9]

# to visualize some images from our data set
fig, axes = plt.subplots(3, 3, figsize=(2*3,2*3))
for i in range(9):
    ax = axes[i//3, i%3]
    ax.imshow(images[i], cmap='gray')
plt.show()




In [ ]:
#Setting up our model

model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=train_seedlings.image_shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(rate=0.15), 
    
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.10),
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.15),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    # hidden layer
    
    tf.keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),    #adding batch normalization
    keras.layers.Dropout(rate=0.10),
  
    
    # 12 output neurons for the 12 classes of Seedling Images
    tf.keras.layers.Dense(12, activation='softmax')
    
    
    ])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])

In [ ]:
## Running our model for 12 epochs


#Model fitting for a number of epochs
history = model.fit_generator(
      train_seedlings,
      steps_per_epoch=44,
      epochs=12,
      validation_data = validation_seedlings,
      validation_steps = 15,
      verbose=1)

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']
    

# returns accuracy of training
print("Training Accuracy:"), print(history.history['acc'][-1])
print("Testing Accuracy:"), print (history.history['val_acc'][-1])

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()
for i, met in enumerate(['acc', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])